In [86]:
import numpy as np
from pcdiff import knn_graph
import open3d
import os

In [87]:
def reduce_points(remove_precentage, pointcloud):
    Num_points = pointcloud.shape[0]
    Num_remove = round(Num_points * remove_precentage)
    Delete_rows = np.random.choice(Num_points, Num_remove, replace=False)
    reduced_cloud = np.delete(pointcloud, Delete_rows, axis=0)
    return reduced_cloud

In [88]:
def delete_areas(area_size, n, pointcloud):
    edge_index = knn_graph(pointcloud, area_size)
    __, col = edge_index
    Num_points = pointcloud.shape[0]    
    Delete_centerpoints = np.random.choice(Num_points, n, replace=False)
    Delete_col = (Delete_centerpoints[:, None] * area_size + np.arange(area_size)).flatten()
    #edge_index_areas = np.array(edge_index_areas.flatten())
    Delete_points = col[Delete_col]
    cloud_with_holes = np.delete(pointcloud, Delete_points, axis=0)
    return cloud_with_holes

In [119]:
def PCQ_reduction(remove_percentage, area_size, n, cloud_name="bevel_gear.xyz", save_to_file=False):
    cwd = os.getcwd()
    cloud_path = os.path.join(cwd,"SelfGeneratedClouds", cloud_name)
    pointcloud = np.loadtxt(cloud_path)[:, 0:3]

    reduced_cloud = reduce_points(remove_percentage, pointcloud)
    cloud_with_holes = delete_areas(area_size, n, reduced_cloud)

    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(cloud_with_holes)
    open3d.visualization.draw_geometries([pcd])

    if save_to_file:
        filename = os.path.join("./SelfGeneratedClouds/Reduced_PCQ_clouds", f"{cloud_name[:-4]}_PDel{remove_percentage}_ADel{n}x{area_size}.xyz")
        np.savetxt(filename, cloud_with_holes, fmt="%.6f", delimiter=" ")

    print('Number of points deleted from',cloud_name,':\n',pointcloud.shape[0] - cloud_with_holes.shape[0])
    
    return 

In [120]:
PCQ_reduction(0.5, 100, 10, cloud_name="bevel_gear.xyz", save_to_file=False)

Number of points deleted from bevel_gear.xyz :
 26961
